In [1]:
import pandas as pd
import numpy as np


In [4]:
# Creating the class Node

class Node:
    def __init__(self, feature=None, split=None, left=None, right=None, label=None):
        self.feature = feature
        self.split = split
        self.left = left
        self.right = right
        self.label = label

In [5]:
# Creating the class Decision Tree
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
        
    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))
        
        if depth == self.max_depth or num_samples < self.min_samples_split or num_labels == 1:
            label = self._most_common_label(y)
            return Node(label=label)
        
        best_feature, best_split = self._find_best_split(X, y, num_samples, num_features)
        left_idx = np.where(X[:, best_feature] < best_split)
        right_idx = np.where(X[:, best_feature] >= best_split)
        left = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right = self._build_tree(X[right_idx], y[right_idx], depth + 1)
        
        return Node(feature=best_feature, split=best_split, left=left, right=right)
    
    def _find_best_split(self, X, y, num_samples, num_features):
        best_gini = float("inf")
        best_feature = None
        best_split = None
        
        for feature in range(num_features):
            feature_values = X[:, feature]
            possible_splits = np.unique(feature_values)
            for split in possible_splits:
                left_idx = np.where(feature_values < split)
                right_idx = np.where(feature_values >= split)
                left_labels = y[left_idx]
                right_labels = y[right_idx]
                gini = self._gini_index(left_labels, right_labels, num_samples)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_split = split
                    
        return best_feature, best_split
    
    def _gini_index(self, left_labels, right_labels, num_samples):
        p_left = len(left_labels) / num_samples
        p_right = len(right_labels) / num_samples
        gini_left = 1.0 - sum((np.unique(left_labels, return_counts=True)[1] / len(left_labels))**2)
        gini_right = 1.0 - sum((np.unique(right_labels, return_counts=True)[1] / len(right_labels))**2)
        gini = p_left * gini_left + p_right * gini_right
        return gini
    
    def _most_common_label(self, y):
        labels, counts = np.unique(y, return_counts=True)
        return labels[np.argmax(counts)]
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])
    
    def _traverse_tree(self, x, node):
        if node.label is not None:
            return node.label
        if x[node.feature] < node.split:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

In [6]:
# Model checking

X = np.array([[1, 2], [3, 4], [5, 6]])
y = np.array([2, 4, 6])
model = DecisionTree()
model.fit(X, y)
y_pred = model.predict(X)
print(y_pred)

[2 4 6]
